In [1]:
def poly_first_order(BT11450):
    # Coefficients
    a0_1dg=274.48668878566474
    a1_1dg=1.07845305
    
    SST_1dg = a0_1dg + a1_1dg*(BT11450-273.15)
    
    return SST_1dg-273.15

def poly_second_order(BT11450):
    # Coefficients
    a0_2dg=265.76204784786813
    a1_2dg=2.06564202
    a2_2dg=-0.02680456
    
    SST_2dg = a0_2dg + a1_2dg*(BT11450-273.15) + a2_2dg*((BT11450-273.15)**2)
    
    return SST_2dg-273.15

def poly_third_order(BT11450):
    # Coefficients
    a0_3dg=315.1038470114878
    a1_3dg=-6.27971675
    a2_3dg=0.43218497
    a3_3dg=-0.00821821
    
    SST_3dg = a0_3dg + a1_3dg*(BT11450-273.15) + a2_3dg*((BT11450-273.15)**2) + a3_3dg*((BT11450-273.15)**3)
    
    return SST_3dg-273.15

def merchant_single_band(BT11450, F):
    a1_1B=-7.461366956846803
    a2_1B=-165.99866494
    a5_1B=1.03440957
    a6_1B=0.57872213
    
    SST_1Band = a1_1B + a2_1B*F + (a5_1B + a6_1B*F)*BT11450
    
    return SST_1Band-273.15

def merchant_dual_band(BT3800, BT11450, F):
    a1_2B=2.4363318540526393
    a2_2B=14.22963516
    a3_2B=1.19597827
    a5_2B=-0.10500926
    a4_2B=-0.19093914
    a6_2B=0.05740321
    
    SST_2Band = a1_2B + a2_2B*F + (a3_2B + a4_2B*F)*BT3800 + (a5_2B + a6_2B*F)*BT11450
    
    return SST_2Band-273.15

In [2]:
# This script runs on "Sentinel1SAR" environment
import os
from pathlib import Path
import numpy as np
import netCDF4 as nc 
from netCDF4 import Dataset
import xarray as xr
from rasterio.transform import from_origin
import rasterio
from pyproj import CRS
from pyproj import Transformer
from pyresample import geometry, kd_tree

In [28]:
dirname='/home/data/sst/Forest-2_NetCDFs'
fname='F002_L1__IR__L2L1M0__2025-06-13T123914.034251Z_2025-06-17T094908.234582Z_ec907996.nc4'

inp_nc=os.path.join(dirname,fname)
nc_path = Path(inp_nc)
ds = xr.open_dataset(nc_path)

lat=ds['lat'].values
lon=ds['lon'].values
SZA=ds['sensor_zenith_angle'].values
BT=ds['brightness_temperature'].values

In [29]:
# Retrieve bands (3.8μm, 11.45μm)
BT3800=np.squeeze(BT[:,:,2])
BT11450=np.squeeze(BT[:,:,0])

# ######################### Polynomial Model ##########################
# # 1st Order Model
SST_1dg = poly_first_order(BT11450)

# # 2nd Order Model
SST_2dg = poly_second_order(BT11450)

# # 3rd Order Model
SST_3dg = poly_third_order(BT11450)

# ######################### Merchant Model ##########################
F=(1/(np.cos(3.14*SZA[:,:,0]/180)))-1
# ######################### Single-Band ##########################
SST_1Band = merchant_single_band(BT11450, F)

# ######################### Dual-Band ##########################
SST_2Band = merchant_dual_band(BT3800, BT11450, F)

In [33]:
# Define which SST method to save as tiff
bt = SST_2Band

nodata = -999

# Define swath geometry (source data)
swath = geometry.SwathDefinition(lons=lon, lats=lat)

# Define target area in EPSG:2100
# Choose resolution in meters (e.g. 1000 m)
res =  150
xmin, xmax = lon.min(), lon.max()
ymin, ymax = lat.min(), lat.max()

# Get projected extent in EPSG:2100
crs_src = CRS.from_epsg(4326)
crs_dst = CRS.from_epsg(2100)

# Transform extent
transformer = Transformer.from_crs(crs_src, crs_dst, always_xy=True)
x_min, y_min = transformer.transform(xmin, ymin)
x_max, y_max = transformer.transform(xmax, ymax)

# Build target grid
nx = int((x_max - x_min) / res)
ny = int((y_max - y_min) / res)

area_def = geometry.AreaDefinition(
    "target_epsg2100",
    "Reprojected Area",
    "epsg2100",
    crs_dst.to_wkt(),
    nx,
    ny,
    (x_min, y_min, x_max, y_max)
)

# Resample swath to grid (nearest or bilinear)
bt_grid = kd_tree.resample_nearest(
    swath, bt, area_def,
    radius_of_influence=600,  # meters
    fill_value=nodata
)

# --- Save to GeoTIFF ---
tiff_suffix = "_merchant2.tiff"
out_tiff = Path(os.path.join(dirname, os.path.splitext(fname)[0] + tiff_suffix))
transform = from_origin(x_min, y_max, res, res)
with rasterio.open(
    out_tiff,
    "w",
    driver="GTiff",
    height=ny,
    width=nx,
    count=1,
    dtype=bt_grid.dtype,
    crs="EPSG:2100",
    transform=transform,
    nodata=nodata
) as dst:
    dst.write(bt_grid, 1)

print(f"Saved: {out_tiff}")

<class 'numpy.ndarray'>
Saved: /home/data/sst/Forest-2_NetCDFs/F002_L1__IR__L2L1M0__2025-06-13T123914.034251Z_2025-06-17T094908.234582Z_ec907996_merchant2.tiff
